In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
class TicTacToe:
    def __init__(self):
        self.board = [0] * 9
        self.current_player = 1
    
    def reset(self):
        self.board = [0] * 9
        self.current_player = 1
        return np.array(self.board, dtype=np.float32)
    
    def get_available_actions(self):
        return [i for i, x in enumerate(self.board) if x == 0]
    
    def make_move(self, action):
        self.board[action] = self.current_player
        if self.check_win(self.current_player):
            return self.current_player, True
        elif self.is_full():
            return 0, True
        else:
            self.current_player *= -1
            return None, False
    
    def check_win(self, player):
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],
            [0, 3, 6], [1, 4, 7], [2, 5, 8],
            [0, 4, 8], [2, 4, 6]
        ]
        for condition in win_conditions:
            if all(self.board[i] == player for i in condition):
                return True
        return False
    
    def is_full(self):
        return all(x != 0 for x in self.board)

    def render(self):
        for i in range(0,9,3):
            print(self.board[i:i+3])
        print()

In [3]:
class PolicyGradientAgent:
    def __init__(self, lr=0.01, gamma=0.99):
        self.gamma = gamma
        self.model = nn.Sequential(
            nn.Linear(9, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 9),
            nn.Softmax(dim=-1)
        )
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.states, self.actions, self.rewards = [], [], []
    
    def choose_action(self, state, available_actions, train=True):
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        action_probs = self.model(state).detach().numpy().flatten()
        if not train: print(action_probs)
        action_probs = np.array([action_probs[i] if i in available_actions else 0 for i in range(9)])
        # action_probs /= np.sum(action_probs)
        # return np.random.choice(9, p=action_probs)
        return np.argmax(action_probs)
    
    def store_outcome(self, state, action, reward):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
    
    def train(self):
        discounted_rewards = self.compute_discounted_rewards()
        states = torch.tensor(self.states, dtype=torch.float32)
        actions = torch.tensor(self.actions, dtype=torch.int64)
        rewards = torch.tensor(discounted_rewards, dtype=torch.float32)
        
        logits = self.model(states)
        action_masks = torch.nn.functional.one_hot(actions, num_classes=9)
        log_probs = torch.sum(action_masks * torch.log(logits + 1e-10), dim=1)
        loss = -torch.mean(log_probs * rewards)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        self.states, self.actions, self.rewards = [], [], []
    
    def compute_discounted_rewards(self):
        discounted_rewards = np.zeros_like(self.rewards, dtype=np.float32)
        cumulative_reward = 0
        for i in reversed(range(len(self.rewards))):
            cumulative_reward = self.rewards[i] + self.gamma * cumulative_reward
            discounted_rewards[i] = cumulative_reward
        return (discounted_rewards - np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-10)

In **compute_discounted_rewards**:

- $G_t = R_t + \gamma G_{t+1}$ or $G_t = R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + ...$
- $\hat{G_t} = \dfrac{G_t - \mu_G}{\sigma_G + \epsilon}$ to reduce variance

In **train**:

- $L(\theta) = −\mathbb{E}\left[G_t \cdot \log \pi_\theta (a_t | s_t)\right] = -\dfrac{1}{N} \sum_{t=1}^{T}G_t \log \pi_\theta (a_t | s_t)$
- Finally, the update will be: $\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)$ where $\nabla_\theta J(\theta) = \sum_{t=1}^{T}G_t \nabla_\theta \log \pi_\theta (a_t | s_t)$

In [4]:
def train(agent, env, both, num_episodes=5000):
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        while not done:
            available_actions = env.get_available_actions()
            if both == False and env.current_player == -1:
                action = np.random.choice(available_actions)
            else:
                action = agent.choose_action(state, available_actions)
            reward, done = env.make_move(action)
            agent.store_outcome(state, action, reward if reward is not None else 0)
            state = np.array(env.board, dtype=np.float32)
        agent.train()
        if (episode + 1) % 500 == 0:
            print(f"Episode {episode + 1}/{num_episodes} completed.")

In [7]:
def evaluate(agent, env, num_games=100, render=False):
    wins, losses, draws = 0, 0, 0
    for _ in range(num_games):
        state = env.reset()
        done = False
        if render: env.render()
        while not done:
            available_actions = env.get_available_actions()
            action = agent.choose_action(state, available_actions, True)
            reward, done = env.make_move(action)
            state = np.array(env.board, dtype=np.float32)
            if render: env.render()
        if reward == 1:
            wins += 1
            if render: print("WIN!")
        elif reward == -1:
            losses += 1
            if render: print("LOSE!")
        else:
            draws += 1
            if render: print("DRAW!")
        if render: print("-----------------------")
    print(f"Evaluation Results: Wins: {wins}, Losses: {losses}, Draws: {draws}")

## Train both

In [5]:
env = TicTacToe()
agent = PolicyGradientAgent()
train(agent, env, both=True)

<ipython-input-3-01615867228a>:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  states = torch.tensor(self.states, dtype=torch.float32)


Episode 500/5000 completed.
Episode 1000/5000 completed.
Episode 1500/5000 completed.
Episode 2000/5000 completed.
Episode 2500/5000 completed.
Episode 3000/5000 completed.
Episode 3500/5000 completed.
Episode 4000/5000 completed.
Episode 4500/5000 completed.
Episode 5000/5000 completed.


In [8]:
evaluate(agent, env)

Evaluation Results: Wins: 0, Losses: 0, Draws: 100


## Train one player

In [9]:
env = TicTacToe()
agent = PolicyGradientAgent()
train(agent, env, both=False)

Episode 500/5000 completed.
Episode 1000/5000 completed.
Episode 1500/5000 completed.
Episode 2000/5000 completed.
Episode 2500/5000 completed.
Episode 3000/5000 completed.
Episode 3500/5000 completed.
Episode 4000/5000 completed.
Episode 4500/5000 completed.
Episode 5000/5000 completed.


In [10]:
evaluate(agent, env)

Evaluation Results: Wins: 100, Losses: 0, Draws: 0
